# Select all the players in the MLB and NPB who batted the same number of times in the same year

In [5]:
%%bigquery 
create or replace view reporting.v_Games_Batting as
select distinct a.yearID, concat(nameFirst,' ' , nameLast) as MLB_Player, concat(j.FName,' ', j.LName) as NPB_Player, j.g as games_batting
from `extended-ascent-252921.seanlahman_modeled.player_stats_all`
 a join `extended-ascent-252921.seanlahman_modeled.players_Beam` b
on a.playerID = b.playerID, `extended-ascent-252921.seanlahman_modeled.player_stats_all` join `extended-ascent-252921.sabermetrics_modeled.jBat_Beam_DF` j
on (a.yearID = j.Year and a.b_g = j.g)
where a.b_g > 50 
order by a.yearID

""


# Averages the number of runs each team in the MLB and NPB scored every year from 1990 to 2006

In [7]:
%%bigquery 
create or replace view reporting.v_MLB_vs_NPB as
select  yearID, a.teamID, "USA" as Country, avg (a.b_R) as average_team_runs 
from `extended-ascent-252921.seanlahman_modeled.player_stats_all` a
where yearID > 1990 and yearID < 2006
group by  a.teamID, yearID
union distinct
select  Year, b.TeamID, "Japan" as Country, avg (b.R) as average_team_runs
from `extended-ascent-252921.sabermetrics_modeled.jBat_Beam_DF` b
where Year > 1990 and Year < 2006
group by  b.TeamID, Year
order by average_team_runs desc 

""


# Finds the players who played in both the MLB and NPB who scored at least one homerun in both leages and the years they played for the different leagues.

In [13]:
%%bigquery 
create or replace table sabermetrics_modeled.test as
select distinct a.yearID as year_MLB, j.Year as year_NPB, concat(nameFirst,' ' , nameLast) as Player, a.b_HR as Homeruns_MLB, j.HR as Homeruns_NPB
from `extended-ascent-252921.seanlahman_modeled.player_stats_all`
 a join `extended-ascent-252921.seanlahman_modeled.players_Beam` b
on a.playerID = b.playerID, `extended-ascent-252921.seanlahman_modeled.player_stats_all` join `extended-ascent-252921.sabermetrics_modeled.jBat_Beam_DF` j
on (concat(nameFirst,' ' , nameLast) = concat(j.FName,' ', j.LName))
where a.b_HR > 0 and j.HR > 0
order by a.yearID

""


In [15]:
%%bigquery 
create or replace view reporting.v_Switchers as
select year_MLB as year, Player, Homeruns_MLB as Homeruns, "USA" as Country
from `extended-ascent-252921.sabermetrics_modeled.test`
union distinct 
select year_NPB as year, Player, Homeruns_NPB as Homeruns, "Japan" as Country
from `extended-ascent-252921.sabermetrics_modeled.test`
order by year

""
